In [4]:
!pip install python-dotenv
from google.colab import drive
drive.mount('/content/drive')
!pip install kaggle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from google.colab import files
uploaded = files.upload()

# Save into ~/.kaggle
import os
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded['kaggle.json'])

!chmod 600 /root/.kaggle/kaggle.json

KeyError: 'kaggle.json'

In [ ]:
!kaggle datasets download -d naelaqel/synthetic-e-commerce-relational-dataset -p '/content/drive/MyDrive/ecom_data'
!unzip '/content/drive/MyDrive/ecom_data/*.zip' -d '/content/drive/MyDrive/ecom_data'

Dataset URL: https://www.kaggle.com/datasets/naelaqel/synthetic-e-commerce-relational-dataset
License(s): MIT
 99% 1.00G/1.00G [00:07<00:00, 139MB/s]
100% 1.00G/1.00G [00:07<00:00, 137MB/s]
Archive:  /content/drive/MyDrive/ecom_data/synthetic-e-commerce-relational-dataset.zip
  inflating: /content/drive/MyDrive/ecom_data/csv/customers.csv  
  inflating: /content/drive/MyDrive/ecom_data/csv/order_items.csv  
  inflating: /content/drive/MyDrive/ecom_data/csv/orders.csv  
  inflating: /content/drive/MyDrive/ecom_data/csv/product_reviews.csv  
  inflating: /content/drive/MyDrive/ecom_data/csv/products.csv  
  inflating: /content/drive/MyDrive/ecom_data/parquet/customers.parquet  
  inflating: /content/drive/MyDrive/ecom_data/parquet/order_items.parquet  
  inflating: /content/drive/MyDrive/ecom_data/parquet/orders.parquet  
  inflating: /content/drive/MyDrive/ecom_data/parquet/product_reviews.parquet  
  inflating: /content/drive/MyDrive/ecom_data/parquet/products.parquet  


In [6]:
!pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.1 MB/s eta 0:00:00


In [8]:
import snowflake.connector
from dotenv import load_dotenv, find_dotenv
load_dotenv()
SF_ACCOUNT   = os.environ['SF_ACCOUNT']
SF_USER      = os.environ['SF_USER']
SF_PASSWORD  = os.environ['SF_PASSWORD']
SF_WAREHOUSE = "ETL_XS"
SF_DATABASE  = "ECOMMERCE"
SF_SCHEMA    = "RAW"

ctx = snowflake.connector.connect(
    user=SF_USER,
    password=SF_PASSWORD,
    account=SF_ACCOUNT,
    warehouse=SF_WAREHOUSE,
    database=SF_DATABASE,
    schema=SF_SCHEMA,
)

cs = ctx.cursor()
print("Connected!")


Connected!


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
from pathlib import Path

csv_dir = "/content/drive/MyDrive/ecom_data/csv"

files = [f for f in os.listdir(csv_dir) if f.endswith(".csv")]
files

['customers.csv',
 'order_items.csv',
 'orders.csv',
 'product_reviews.csv',
 'products.csv']

In [24]:
for file in files:
    full_path = str(Path(csv_dir) / file)
    table_name = file.replace(".csv","").upper()

    print(f"Uploading {file} -> @{table_name}")

    # 1) Upload file to table stage
    cs.execute(f"PUT file://{full_path} @%{table_name} OVERWRITE=TRUE")

    # 2) Load into real table using named file format
    cs.execute(f"""
        COPY INTO RAW.{table_name}
        FROM @%{table_name}
        FILE_FORMAT = (FORMAT_NAME = 'CSV_FORMAT')
        ON_ERROR = 'CONTINUE'
    """)


Uploading customers.csv -> @CUSTOMERS
Uploading order_items.csv -> @ORDER_ITEMS
Uploading orders.csv -> @ORDERS
Uploading product_reviews.csv -> @PRODUCT_REVIEWS
Uploading products.csv -> @PRODUCTS


In [11]:
cs.execute("LIST @%CUSTOMERS")
print(cs.fetchall())


[(datetime.datetime(2025, 11, 16, 19, 31, 23, 708000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>),
  'CUSTOMERS',
  'ECOMMERCE',
  'RAW',
  'TABLE',
  '',
  '',
  0,
  0,
  'ACCOUNTADMIN',
  '1',
  'OFF',
  'OFF',
  'OFF',
  None,
  None,
  'N',
  'N',
  'ROLE',
  'N',
  'N',
  'N',
  'N',
  'N'),
 (datetime.datetime(2025, 11, 16, 19, 34, 52, 310000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>),
  'ORDERS',
  'ECOMMERCE',
  'RAW',
  'TABLE',
  '',
  '',
  0,
  0,
  'ACCOUNTADMIN',
  '1',
  'OFF',
  'OFF',
  'OFF',
  None,
  None,
  'N',
  'N',
  'ROLE',
  'N',
  'N',
  'N',
  'N',
  'N'),
 (datetime.datetime(2025, 11, 16, 19, 31, 50, 371000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>),
  'ORDER_ITEMS',
  'ECOMMERCE',
  'RAW',
  'TABLE',
  '',
  '',
  0,
  0,
  'ACCOUNTADMIN',
  '1',
  'OFF',
  'OFF',
  'OFF',
  None,
  None,
  'N',
  'N',
  'ROLE',
  'N',
  'N',
  'N',
  'N',
  'N'),
 (datetime.datetime(2025, 11, 16, 19, 

In [13]:
import os
print(csv_dir)
print(os.listdir(csv_dir))


/content/drive/MyDrive/ecom_data/csv
['customers.csv', 'order_items.csv', 'orders.csv', 'product_reviews.csv', 'products.csv']
